In [1]:
### ICI implementation in python (Global Driver Search using matrix operations): Jinling Liu 12/26/2021
### Can be applied to multiple traits or single trait; ready to be run on server after converted to .py file.
### 02-09-22 updated the function of lgM_cal_1 so it can either calculate using topGD or sGD, with a flag of "use_topGD" before that function

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2,venn2_circles
import json
import os
import shutil
import scipy.stats as stats
from scipy.stats import fisher_exact
import scipy
from datetime import datetime
import math
import time
from joblib import Parallel, delayed


In [40]:
def read_traitsF (traitsF):  ### read the .csv file with traits (subjects_row x traits_column)
    traits = pd.read_csv(traitsF,index_col = 0) # make sure the subIDs become the index column; as default, column names are inferred from the first line of the file
    subIDs = list(traits.index)
    traitIDs = traits.columns
    traits = np.array(traits,dtype=np.int16) #np.int8 has changed the type to int8; when using int8, the subIDs become negative.
#     print(np.shape(traits))
#     print(np.sum(traits)) # sum gives odd result of -94.
    return (subIDs, traitIDs, traits) # list, array

In [41]:
def read_variantsF (variantsF):## read the large genomic file (row_SNPs x column_subjects) line by line
    header = True
    variants = []
    with open(variantsF) as f:
        for line in f: # go through each SNP
            if header: # extract the head of subject IDs
                subIDs = line.strip().split(',')
                subIDs = list(subIDs[1:])
                header = False
                continue
            line = line.strip().split(',') #vID = line[0] ,the variant ID; line1 = line[1:] ,the SNP state values for all the subjects
            variants.append(line)
    #print(len(variants))
    variants = np.array(variants)
    varIDs = list(variants[:,0])
    #print(varIDs)
    variants = variants[:,1:].astype(int) # int or np.int8 does not work and variants stay in the type of float 64.??
    A0 = np.ones(len(subIDs))
    variants = np.row_stack((A0,variants))
    varIDs.insert(0,'A0')
    #print(np.shape(variants))
    return (subIDs, varIDs, variants) # list, list and array

In [42]:
def read_variantsF1 (variantsF):## read the large genomic file (row_SNPs x column_subjects) using pandas
    df = pd.read_csv(variantsF,index_col = 0) # this will turn the first column of varIDs into index thus df.columns will only include subIDs
    varIDs = list(df.index)
    subIDs = list(int(x) for x in df.columns)
    variants = np.array(df,dtype=np.int8) # Somehow, np.int8 does not work here.
    A0 = np.ones(len(subIDs),dtype=np.int8)
    variants = np.row_stack((A0,variants))
    varIDs.insert(0,'A0')
    df = pd.DataFrame(variants,index=varIDs,columns=subIDs,dtype=np.int8)
    #print(np.shape(variants))
    return (subIDs, varIDs, variants, df) # list, list, array and dataframe

In [43]:
def DriverSearch (traits,variants): 
    ### Calcuate and return the lgM for all the drivers or any driver for any given population for multiple traits
    ### Get the top/bottom GD and SD as well as their lgM; this can be done in the lgM_cal function so this function can stay the same 
    ### Get the nxk matrix of traits==0 and the nxk matrix of traits==1 (n,#subjects;k,#traits; thus capable of working with multipe traits)
    ### if no individuals are in V0 group when the passed variants is [], the V0D0 counts as well as lgM will be 0; the max value/index are both turned as 0
    ### no other SNPs have a constant value since those have been removed in the preprocessing step
    
#     print(variants.shape) #(1442,) if only variant is passed in.
#     print(variants.ndim)
    
    bpMask0 = traits==0
    bpMask0 = bpMask0.astype(np.int16)
    d0 = np.sum(bpMask0)
    #print(d0) #930 HTN and 4360 non-HTN making a totla of 5290 subjects
    bpMask1 = traits==1
    bpMask1 = bpMask1.astype(np.int16)
    d1 = np.sum(bpMask1)
    #print(d1) #930 HTN and 4360 non-HTN making a totla of 5290 subjects
    ### Get the mxn vector of snp==0 and the mxn vector of snp==1
    snpMask0 = variants==0
    snpMask0 = snpMask0.astype(np.int16)
    #print(snpMask0.sum(axis=1))

    snpMask1 = variants==1
    snpMask1 = snpMask1.astype(np.int16)
    #print(snpMask1.sum(axis=1))
#     print(snpMask1.shape)
#     print(bpMask1.shape)
    
    ### Get the four mx1 vector as below: m is # of SNPs in the dataset; for each SNP, the corresponding 4 values 
    ### from the 4 vectors make up the 2x2 tables between SNP and hypertension
    V0D0 = snpMask0@bpMask0 #snpMask0, variants_row x subjects_column
    V1D0 = snpMask1@bpMask0 #bpMask0, subjects_row x traits_column
    V0D1 = snpMask0@bpMask1
    V1D1 = snpMask1@bpMask1
#     print(V0D0)
#     print(V0D1)

    # Calculate the log Marginal LikelihooD for this particular SNP based on the collected counts and equation 5 in the worD file
    
    # when j=0 (V=0)
    lgM = scipy.special.loggamma(2.0) - scipy.special.loggamma(2.0+V0D1+V0D0)
    lgM += scipy.special.loggamma(1.0+V0D0) - scipy.special.loggamma(1.0)
    lgM += scipy.special.loggamma(1.0+V0D1) - scipy.special.loggamma(1.0)
    
    # when j=1 (V=1)
    lgM += scipy.special.loggamma(2.0) - scipy.special.loggamma(2.0+V1D1+V1D0)
    lgM += scipy.special.loggamma(1.0+V1D0) - scipy.special.loggamma(1.0)
    lgM += scipy.special.loggamma(1.0+V1D1) - scipy.special.loggamma(1.0)
    
    
    if variants.ndim == 1:
#         print('hi')
        lgM = lgM.reshape(1,lgM.shape[0]) # lgM is #traits x 1
    
#     print(np.shape(lgM)) # lgM is a 2D array of #variants x #traits
    
#     array([[-3127.91831177, -3134.36270821], an example of a 2-D array with two [] rather than [] for 1-D array
#        [-3130.8809211 , -3137.44571946]]) 
#     max_value = np.max(lgM,axis=0) # get the max_value and max_index among all the variants (across all the rows) for each trait on the column
#     max_index = np.argmax(lgM,axis=0)
           
#     min_value = np.min(lgM,axis=0) # get the max and index for each row of trait along the column of the 2-D array
#     min_index = np.argmin(lgM,axis=0)
    
#     SD_value = lgM[i,0]
#     topGD_value = lgM[9980,0] #array([9980]),'ss74821773_G', is the topGD for HTN in the 5290 x 38K file
    
#     return (max_value[0],max_index[0],min_value[0],min_index[0],SD_value,i,topGD_value) # scaler for each of the returned element, since it only takes the value for the fir 
#     return (max_value, max_index) # either scaler or array for each of the returned element 
    return(lgM)

In [44]:
def GDsearch_all(traits,variants): 
## Get all the stats for all the variants in any given population for multiple traits; particulary used for the entire population
## Get the nxk matrix of traits==0 and the nxk matrix of traits==1 (n,#subjects;k,#traits)
## if no individuals are in V0 group when the passed variants is [], the V0D0 counts as well as lgM will be 0.
    
#     print(variants.shape) #(1442,) if only variant is passed in.
#     print(variants.ndim)
    
    bpMask0 = traits==0
    bpMask0 = bpMask0.astype(np.int16)
    d0 = np.sum(bpMask0)
    #print(d0) #930 HTN and 4360 non-HTN making a totla of 5290 subjects
    bpMask1 = traits==1
    bpMask1 = bpMask1.astype(np.int16)
    d1 = np.sum(bpMask1)
    #print(d1) #930 HTN and 4360 non-HTN making a totla of 5290 subjects
    ### Get the mxn vector of snp==0 and the mxn vector of snp==1
    snpMask0 = variants==0
    snpMask0 = snpMask0.astype(np.int16)
    #print(snpMask0.sum(axis=1))

    snpMask1 = variants==1
    snpMask1 = snpMask1.astype(np.int16)
    #print(snpMask1.sum(axis=1))
#     print(snpMask1.shape)
#     print(bpMask1.shape)
    
    ### Get the four mx1 vector as below: m is # of SNPs in the dataset; for each SNP, the corresponding 4 values 
    ### from the 4 vectors make up the 2x2 tables between SNP and hypertension
    V0D0 = snpMask0@bpMask0
    V1D0 = snpMask1@bpMask0
    V0D1 = snpMask0@bpMask1
    V1D1 = snpMask1@bpMask1
#     print(V0D0)
#     print(V0D1)
    #print(V1D1.dtype)
#     # GiVen the Dirichlet Distributions we are using, the expectation of these conDitional probabilities is as follows:
    cp_D1V1 = (1 + V1D1)/(2 + V1D1 + V1D0)*1.0 # P(D=1|V=1) = (alpha11 + V1D1)/(alpha1 + V1D1 + V1D0)*1.0                   
    cp_D1V0 = (1 + V0D1)/(2 + V0D1 + V0D0)*1.0  #P(D=1|V=0) = (alpha01 + V0D1)/(alpha0 + V0D1 + V0D0)*1.0              
    RR = cp_D1V1/cp_D1V0 #RR is risk ratio; OR is oDDs ratio
#     OR = 1.0*V1D1*V0D0/V1D0/V0D1
#     print(OR)
    # Calculate the log Marginal LikelihooD for this particular SNP based on the collected counts and equation 5 in the worD file
    # when j=0 (V=0)
    lgM = scipy.special.loggamma(2.0) - scipy.special.loggamma(2.0+V0D1+V0D0)
    lgM += scipy.special.loggamma(1.0+V0D0) - scipy.special.loggamma(1.0)
    lgM += scipy.special.loggamma(1.0+V0D1) - scipy.special.loggamma(1.0)
    #print(lgM)
    # when j=1 (V=1)
    lgM += scipy.special.loggamma(2.0) - scipy.special.loggamma(2.0+V1D1+V1D0)
#     print(lgM)
#     print(lgM.shape)
    lgM += scipy.special.loggamma(1.0+V1D0) - scipy.special.loggamma(1.0)
    lgM += scipy.special.loggamma(1.0+V1D1) - scipy.special.loggamma(1.0)
    
#     print(np.shape(lgM))
#     print(lgM)
    
    if variants.ndim == 1:
#         print('hi')
        lgM = lgM.reshape(1,lgM.shape[0]) # lgM is #traits x 1
    max_value = np.max(lgM,axis=0) # get the max and index for each row of trait along the column of the 2-D array
    max_index = np.argmax(lgM,axis=0)
#     print(max_index)
#     print(max_value)
#     print(lgM)
#     lgsum = scipy.special.logsumexp(np.array(lgMwA0)) # returned log(M1+M2+...)
#     print(lgsum)
#     PP = np.exp((np.array(lgMwA0)-lgsum)) # returned the posterial probability list by dividing the marginal with the sum of the marginals
#     print(sum(PP))
#     pp_A0 = PP[-1] # the last element in PP
#     A0 = np.array([d1,d0,0,0,cp_d1v1_A0,'NA','NA','NA',lgM_A0,pp_A0])
#     print(np.shape(A0))
    
#     snpStat = np.vstack((V1D1,V1D0, V0D1, V0D0, cp_D1V1, cp_D1V0, RR, OR, lgM, PP[:-1])) 
#     print(np.shape(snpStat))
#     snpStat = np.column_stack((snpStat, A0)) # hstack requires the same dimension between the two arrays
#     print(np.shape(snpStat))
#     snpStat = np.transpose(snpStat)
#     print(np.shape(snpStat))
    
    return (RR,lgM,max_value,max_index)

In [45]:
def lgMcal(varID): ## use DriverSearch for V0 and SD_lgM_V1 for V1
#     i = varIDs_str.index(varID) #when bigRR is used
    i = varIDs.index(varID) #when varIDs is used in the last paralleing code
    #print(i) # not sure why this would not print if using the above parallel code
    index1 = variants[i,:]==1
    index0 = variants[i,:]==0
    
#     V1 = variants[i,index1] # 1 SNP X 2478 subjects who have v=1 for this SNP i
    V1 = variants[:,index1]
#     V1wTopGD = variants[9980,index1] #array([9980]),'ss74821773_G', is the topGD for HTN in the 5290 x 38K file
    
    V0 = variants[:,index0] # V0 will be [] and its shape will be (0,) if index0 is all false
#     V0wA = variants[i,index0]

    BP_V1 = traits[index1] # 2478 subjects' hypertension status who have v=1 for this SNP （2478,)
    BP_V0 = traits[index0] # (5290-2478) subjects' hypertension status who have v=1 for this SNP
#     print(index1.sum())
#     lgM_v1 = SD_lgM_V1(BP_V1) ## Calculate lgM_v1

#     maxValueV1, maxIndexV1, minValueV1, minIndexV1, SDvalueV1, i, topGDvalueV1 = DriverSearch(BP_V1,V1,i)
#     maxValueV0, maxIndexV0, minValueV0, minIndexV0, SDvalueV0, i, topGDvalueV0 = DriverSearch(BP_V0,V0,i)
#     lgM_v1 = SD_lgM_V1(BP_V1)
    lgMv1_SD = DriverSearch(BP_V1,variants[i,index1])[0] # this should be as efficient as SD_lgM_V1; 
    # with [0], the original 2D array, array([[-3127.91831177,...]]), 
    # becomes the format of 1D array, array([-3127.91831177,...]),thus consistent with the other output values

    lgMv0 = DriverSearch(BP_V0,V0) #lgM_v0 is the 2D array containing the lgM for all the variants (row) for all the different traits (column)
    lgMv0_sGD = np.max(lgMv0,axis=0) # get the max and index for each row of trait along the column of the 2-D array
    sGD_index = np.argmax(lgMv0,axis=0)          
#     min_value = np.min(lgM,axis=0) # get the max and index for each row of trait along the column of the 2-D array
#     min_index = np.argmin(lgM,axis=0)
#     SD_value = lgM[i,0]
#     topGD_value = lgM[9980,0] #array([9980]),'ss74821773_G', is the topGD for HTN in the 5290 x 38K file
    sGD = []
    for item in sGD_index:
        sGD.append(varIDs[item])
    sGD = np.array(sGD)
    
    lgMv0_topGD = []
    r = []
    k=0
    for j in topGD_index: # topGD_index is a global variable obtained outside this function
        lgMv0_topGD.append(lgMv0[j,k])
        r1 = stats.spearmanr(variants[i,:],variants[j,:])[0]
        r.append(r1)
        k = k + 1
    lgMv0_topGD = np.array(lgMv0_topGD)
    r = np.array(r)

#     v1GD = varIDs[v1GD_index[0]]
#     V1GDmin = varIDs[v1GDmin_index[0]]
#     constant = np.ones(index1.sum() - 1).tolist()
#     constant.append(1.1) # does not change the r between 1.1 and 1.0000001

    lgM_v1v0 = lgMv1_SD + lgMv0_sGD
    
#     r_GD_SD = stats.spearmanr(variants[i,:],variants[maxIndexV0,:])[0] # default parameters; 
#     # GD refer to the max driver for V0; v1GD refer to the max driver for V1; v1GDmin refer to the min driver for V1.
#     r_GD_v1GD = stats.spearmanr(variants[maxIndexV1,:],variants[maxIndexV0,:])[0] # default parameters
#     r_GD_v1GDmin = stats.spearmanr(variants[minIndexV1,:],variants[maxIndexV0,:])[0] # default parameters
#     r_topGD_SD = stats.spearmanr(variants[i,:],variants[9980,:])[0] # default parameters
#     r_v1_v1GD = stats.spearmanr(variants[v1GD_index[0],:],variants[i,:])[0] # r in the entire population
#     r_v1_v1GD_V1 = stats.pearsonr(variants[v1GD_index[0],index1],constant)[0] # r in the V1 population
#     return(SDvalueV1, topGDvalueV1, maxValueV1, minValueV1, maxValueV0, topGDvalueV0, SDvalueV0, minValueV0, lgM_v1v0, maxIndexV0, sGD, r_GD_SD, r_GD_v1GD, r_GD_v1GDmin, r_topGD_SD, i, varID)
    return(lgMv1_SD, lgMv0_sGD, lgMv0_topGD, lgM_v1v0, sGD, r, i, varID)

In [91]:
def lgMcal_1(varID): ## use DriverSearch for V0 and SD_lgM_V1 for V1 ## designed for using one topGD
#     i = varIDs_str.index(varID) #when bigRR is used
    i = varIDs.index(varID) #when varIDs is used in the last paralleing code
    #print(i) # not sure why this would not print if using the above parallel code
    index1 = variants[i,:]==1
    index0 = variants[i,:]==0
    
#     V1 = variants[i,index1] # 1 SNP X 2478 subjects who have v=1 for this SNP i
    V1 = variants[:,index1]
#     V1wTopGD = variants[9980,index1] #array([9980]),'ss74821773_G', is the topGD for HTN in the 5290 x 38K file
    if use_oneTopGD:
        V0 = variants[topGD_index][:,index0] # we will only consider and search over all the unique topGDs from all the traits;            #thus one topGD for trait1 may be selected as the sGD for trait2.
    else:
        V0 = variants[:,index0] # V0 will be [] and its shape will be (0,) if index0 is all false
#     V0wA = variants[i,index0]

    BP_V1 = traits[index1] # 2478 subjects' hypertension status who have v=1 for this SNP （2478,)
    BP_V0 = traits[index0] # (5290-2478) subjects' hypertension status who have v=1 for this SNP
#     print(index1.sum())
#     lgM_v1 = SD_lgM_V1(BP_V1) ## Calculate lgM_v1

#     maxValueV1, maxIndexV1, minValueV1, minIndexV1, SDvalueV1, i, topGDvalueV1 = DriverSearch(BP_V1,V1,i)
#     maxValueV0, maxIndexV0, minValueV0, minIndexV0, SDvalueV0, i, topGDvalueV0 = DriverSearch(BP_V0,V0,i)
#     lgM_v1 = SD_lgM_V1(BP_V1)
    lgMv1_SD = DriverSearch(BP_V1,variants[i,index1])[0] # this should be as efficient as SD_lgM_V1; 
    # with [0], the original 2D array, array([[-3127.91831177,...]]), 
    # becomes the format of 1D array, array([-3127.91831177,...]),thus consistent with the other output values

    lgMv0 = DriverSearch(BP_V0,V0) #lgM_v0 is the 2D array containing the lgM for all the variants (row) for all the different traits (column)
    lgMv0_topGD = [] # collect the lgMv0_topGD for each trait in a 1D array; the lgM value for V0 group when using topGD as the driver
    r = []           # collect the r between SD and topGD for each trait in a 1D array
    
    if use_oneTopGD:
        for m in range(0,len(traitIDs)):
            lgMv0_topGD.append(lgMv0[m,m])
        for j in topGD_index: # topGD_index is a global variable obtained outside this function
            r1 = stats.spearmanr(variants[i,:],variants[j,:])[0]
            r.append(r1)
        lgMv0_sGD = np.zeros(len(traitIDs))
        sGD = np.zeros(len(traitIDs))
    else:
        lgMv0_sGD = np.max(lgMv0,axis=0) # get the max and index for each row of trait along the column of the 2-D array
        sGD_index = np.argmax(lgMv0,axis=0)          

        sGD = [] # collect the sGD for each trait in a 1D array
        for item in sGD_index:
            sGD.append(varIDs[item])
        sGD = np.array(sGD) 

        k=0  # collect the lgMv0_topGD and r for each trait in a 1D array specifically 
        for j in topGD_index: 
            lgMv0_topGD.append(lgMv0[j,k]) 
            r1 = stats.spearmanr(variants[i,:],variants[j,:])[0]
            r.append(r1)
            k = k + 1
    
    lgMv0_topGD = np.array(lgMv0_topGD) 
    r = np.array(r) 
    

    lgM_v1v0 = lgMv1_SD + lgMv0_sGD
    
#     r_GD_SD = stats.spearmanr(variants[i,:],variants[maxIndexV0,:])[0] # default parameters; 
#     # GD refer to the max driver for V0; v1GD refer to the max driver for V1; v1GDmin refer to the min driver for V1.
#     r_GD_v1GD = stats.spearmanr(variants[maxIndexV1,:],variants[maxIndexV0,:])[0] # default parameters
#     r_GD_v1GDmin = stats.spearmanr(variants[minIndexV1,:],variants[maxIndexV0,:])[0] # default parameters
#     r_topGD_SD = stats.spearmanr(variants[i,:],variants[9980,:])[0] # default parameters
#     r_v1_v1GD = stats.spearmanr(variants[v1GD_index[0],:],variants[i,:])[0] # r in the entire population
#     r_v1_v1GD_V1 = stats.pearsonr(variants[v1GD_index[0],index1],constant)[0] # r in the V1 population
#     return(SDvalueV1, topGDvalueV1, maxValueV1, minValueV1, maxValueV0, topGDvalueV0, SDvalueV0, minValueV0, lgM_v1v0, maxIndexV0, sGD, r_GD_SD, r_GD_v1GD, r_GD_v1GDmin, r_topGD_SD, i, varID)
    return(lgMv1_SD, lgMv0_sGD, lgMv0_topGD, lgM_v1v0, sGD, r, i, varID)